# Traffic sign detection and classification

In [1]:
from xml.dom import minidom
from os import walk
import pandas as pd
import cv2 as cv
import numpy as np
from sklearn import metrics


In [2]:
filenames = next(walk("res/annotations"), (None, None, []))[2]  # [] if no file

signs = []

for annotation in filenames:
    # parse an xml file by name
    file = minidom.parse("res/annotations/" + annotation)

    #use getElementsByTagName() to get tag
    path = "res/images/" + file.getElementsByTagName('filename')[0].firstChild.data
    filename = file.getElementsByTagName('filename')[0].firstChild.data
    name = file.getElementsByTagName('name')[0].firstChild.data
    # truncated = file.getElementsByTagName('truncated')[0].firstChild.data
    # occluded = file.getElementsByTagName('occluded')[0].firstChild.data
    # difficult = file.getElementsByTagName('difficult')[0].firstChild.data

    if name == "trafficlight":
        continue

    signs.append([filename, name, path])

df = pd.DataFrame(signs, columns=['filename', 'name', 'path'])

In [3]:
def condition_classes(s):
    if s['name'] == 'speedlimit':
        return 0
    elif s["name"] == 'crosswalk':
        return 1
    elif s["name"] == "stop":
        return 2


df["class"] = df.apply(condition_classes, axis=1)

In [4]:
def concatenate_and_write(row):
    img = cv.imread(row.path)
    hist = cv.imread("output/histogram/" + row.filename)
    segm = cv.imread("output/segmentation/" + row.filename)
    post = cv.imread("output/post_processing/" + row.filename)

    vis = np.concatenate((img, hist, segm, post), axis=1)

    cv.imwrite("output/concatenated/" + row.filename, vis)

## Step 1 - Histogram equalization

TODO - Need to improve the histogram equalization

In [5]:
def apply_histogram_equalization(row):
    img = cv.imread(row.path)
    lab = cv.cvtColor(img, cv.COLOR_BGR2LAB)

    clahe = cv.createCLAHE(clipLimit=10.0,tileGridSize=(8,8))

    lab[...,0] = clahe.apply(lab[...,0])

    out = cv.cvtColor(lab, cv.COLOR_LAB2BGR)

    cv.imwrite("output/histogram/" + row.filename, out)

In [6]:
df.apply(apply_histogram_equalization, axis=1);

## Step 2 - Segmentation by Color

In [7]:
def apply_segmentation(row):
    img = cv.imread(row.path)
    # TODO - work on histogram equalization
    img_hist = cv.imread("output/histogram/" + row.filename)
    img_hsv = cv.cvtColor(img_hist, cv.COLOR_BGR2HSV)

    lower_red_m1 = (0, 70, 60)
    upper_red_m1 = (10, 255, 255)

    lower_red_m2 = (170, 70, 60)
    upper_red_m2 = (180, 255, 255)

    lower_blue_m3 = (94, 127, 20)
    upper_blue_m3 = (126, 255, 200)

    mask1 = cv.inRange(img_hsv, lower_red_m1, upper_red_m1)
    mask2 = cv.inRange(img_hsv, lower_red_m2, upper_red_m2)
    mask3 = cv.inRange(img_hsv, lower_blue_m3, upper_blue_m3)

    mask = mask1 + mask2 + mask3

    # out = cv.bitwise_and(img_hist, img_hist, mask=mask)

    cv.imwrite("output/segmentation/" + row.filename, mask)

In [8]:
df.apply(apply_segmentation, axis=1);

## Step 3 - Post-Processing

In [9]:
def apply_post_processing(row):
    img = cv.imread(row.path)
    segm_img = cv.imread("output/segmentation/" + row.filename, cv.IMREAD_GRAYSCALE)

    # apply median filter to remove noise
    out = cv.medianBlur(segm_img, 5)
    rows, cols = out.shape

    # Taking a matrix of size 5 as the kernel
    kernel = np.ones((5, 5), np.uint8)

    # morphological operations
    out = cv.morphologyEx(out, cv.MORPH_CLOSE, kernel, iterations=4)

    # remove small and weird objects
    contours, hierarchy = cv.findContours(out, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)[-2:]
    for contour in contours:
        x, y, w, h = cv.boundingRect(contour)
        aspect_ratio = float(w) / h
        if cv.contourArea(contour) < 1 / 1500.0 * rows * cols and (aspect_ratio > 0.5 or aspect_ratio < 1.3):
            out = cv.fillPoly(out, pts=contour, color=(0, 0, 0))

    mask = np.full(img.shape, 0, "uint8")
    contours, hierarchies = cv.findContours(out, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)[-2:]
    for cnt in contours:
        cv.drawContours(mask, [cnt], -1, (255, 255, 255), -1)

    mask = cv.cvtColor(mask, cv.COLOR_BGR2GRAY)
    # morphological operations
    out = cv.erode(mask, kernel, iterations=1)
    out = cv.dilate(mask, kernel, iterations=1)

    cv.imwrite("output/post_processing/" + row.filename, out)

In [10]:
df.apply(apply_post_processing, axis=1);

In [11]:
df.apply(concatenate_and_write, axis=1);

## Step 4 - Shape Recognition

In [12]:
def shape_recognition(row):
    img = cv.imread(row.path)
    processed = cv.imread("output/post_processing/" + row.filename, 0)

    _, thresh = cv.threshold(processed, 240, 255, cv.CHAIN_APPROX_NONE)
    contours, _ = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)[-2:]

    contours = sorted(contours, key=lambda x: -cv.contourArea(x))[:10]

    shapes = []

    for contour in contours:
        if float(cv.contourArea(contour) / (img.shape[0]*img.shape[1])) >= 0.95:
            continue
        approx = cv.approxPolyDP(contour, 0.01*cv.arcLength(contour, True), True)
        if len(approx) == 4:
            shapes.append(("rectangle", cv.contourArea(contour), (cv.boundingRect(contour))))
        elif len(approx) == 8:
            shapes.append(("octagon", cv.contourArea(contour), (cv.boundingRect(contour))))
        elif len(approx) > 8:
            shapes.append(("circle", cv.contourArea(contour), (cv.boundingRect(contour))))

    return shapes

In [13]:
df["shapes"] = df.apply(shape_recognition, axis=1);

In [14]:
def classify(row):
    shapes_info = row.shapes
    if len(shapes_info) == 0:
        return -1

    shapes = [x[0] for x in shapes_info]
    shape = shapes[0]

    if shape == 'circle':
        return 0
    elif shape == 'rectangle':
        return 1
    elif shape == 'octagon':
        return 2

    return -1

df["classification"] = df.apply(classify, axis=1);

## Results

In [15]:
df_found = df[df["classification"] != -1]

accuracy_total = metrics.accuracy_score(df["class"], df["classification"])
accuracy_found = metrics.accuracy_score(df_found["class"], df_found["classification"])

print("Classes:\n", df["class"].value_counts())
print("Detected classes:\n", df["classification"].value_counts())
print("Accuracy Total: {:.02f}%".format(accuracy_total*100))
print("Accuracy Signs Found: {:.02f}%".format(accuracy_found*100))

Classes:
 0    652
1     88
2     76
Name: class, dtype: int64
Detected classes:
  0    716
 2     57
-1     25
 1     18
Name: classification, dtype: int64
Accuracy Total: 78.43%
Accuracy Signs Found: 80.91%


In [16]:
failed_0 = len(df[(df["class"] == 0) & (df["classification"] != 0)])
failed_1 = len(df[(df["class"] == 1) & (df["classification"] != 1)])
failed_2 = len(df[(df["class"] == 2) & (df["classification"] != 2)])

right_0 = len(df[(df["class"] == 0) & (df["classification"] == 0)])
right_1 = len(df[(df["class"] == 1) & (df["classification"] == 1)])
right_2 = len(df[(df["class"] == 2) & (df["classification"] == 2)])


print("Class 0: R:{}|W:{}".format(right_0, failed_0))
print("Class 1: R:{}|W:{}".format(right_1, failed_1))
print("Class 2: R:{}|W:{}".format(right_2, failed_2))

Class 0: R:603|W:49
Class 1: R:12|W:76
Class 2: R:25|W:51
